In [1]:
import os
from dotenv import load_dotenv , find_dotenv

_ =load_dotenv(find_dotenv())


from langchain_groq import ChatGroq

chatm = ChatGroq(
    model="llama3-70b-8192"
)

Runnable Passthrough

. It does not do anything to the input data. Let's see it in a very simple example: a
  chain with just RunnablePassthrough() will output the original input wihout any modification

In [7]:
from langchain_core.runnables import RunnablePassthrough
chain = RunnablePassthrough()

chain.invoke("subodh")

'subodh'

Runnable Lambda

. To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.

In [8]:
from langchain_core.runnables import RunnableLambda

def russian_lastname(name:str):
    return f"{name}ovich"


chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("subodh")

'subodhovich'

Runnable Parallel

used to run the tasks in parallel. most useful runnable.
in the following chain, runnable parallel is going to run these tow tasks in parllel:
    operation_a will use runnablepassthrough
    operation_b will use runnablelambda 

In [10]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a":RunnablePassthrough(),
        "operation_b":RunnableLambda(russian_lastname)
    }
)

chain.invoke("subodh")

{'operation_a': 'subodh', 'operation_b': 'subodhovich'}

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

def russian_lastname_from_dictionay(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "operation_a":RunnablePassthrough(),
        "soccer_player":RunnableLambda(russian_lastname_from_dictionay),
        "operation_b":RunnablePassthrough(),
    }
) | prompt | chatm | output_parser

chain.invoke({"name":"messi","name2":"Abram"
              })


'I think you meant "Messiovich" doesn\'t exist, but I\'m assuming you meant Lionel Messi!\n\nHere\'s a curious fact about Messi:\n\n**Messi\'s growth hormone deficiency**: When Messi was 11 years old, he was diagnosed with a growth hormone deficiency, which meant his body wasn\'t producing enough growth hormone. This condition can affect growth and development in children. FC Barcelona, his future club, agreed to pay for his treatment, which involved daily injections of growth hormone for several years. This treatment helped Messi grow to his adult height of 5 feet 7 inches (170 cm), and the rest, as they say, is history!'

Advance use of Runnable Parrallel

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate   
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_texts(["subodh focusses on work , AI,ML,DL."],embedding=OpenAIEmbeddings())

retreiver = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}
question:{question}"""

prompt = ChatPromptTemplate.from_template(template)

chatm = ChatGroq( model="llama3-70b-8192")

chain= (
    RunnableParallel({"context":retreiver,"question":RunnablePassthrough()})
    | prompt
    | chatm
    | StrOutputParser()
)
chain.invoke("what is subodh?")

Below is the example of itemgetter 

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate   
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_texts(["subodh focusses on work , AI,ML,DL."],embedding=OpenAIEmbeddings())

retreiver = vectorstore.as_retriever()


template = """Answer the question based only on the following context:
{context}
question:{question}

Answer in the following language:{language}"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context":itemgetter("question") |  retreiver,
        "question":itemgetter("question") ,
        "language":itemgetter("language")
    }
    | prompt
    | chatm
    | StrOutputParser()
)

chain.invoke({"question":"what is subodh?","language":"english"})